In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import multiprocessing
import difflib

train = pd.read_csv('../input/train.csv')#[:100]
test = pd.read_csv('../input/test.csv')#[:100]

/home/takaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def extract_6w1h(data):
    for i, row in data.iterrows():
        q1_words = str(row['question1']).lower().split()
        q2_words = str(row['question2']).lower().split()
        
        data.set_value(i, 'q1_what', 'what' in q1_words)
        data.set_value(i, 'q2_what', 'what' in q2_words)
        
        data.set_value(i, 'q1_when', 'when' in q1_words)
        data.set_value(i, 'q2_when', 'when' in q2_words)
        
        data.set_value(i, 'q1_why', 'why' in q1_words)
        data.set_value(i, 'q2_why', 'why' in q2_words)
        
        data.set_value(i, 'q1_where', 'where' in q1_words)
        data.set_value(i, 'q2_where', 'where' in q2_words)
        
        data.set_value(i, 'q1_who', 'who' in q1_words)
        data.set_value(i, 'q2_who', 'who' in q2_words)
        
        data.set_value(i, 'q1_which', 'which' in q1_words)
        data.set_value(i, 'q2_which', 'which' in q2_words)
        
        data.set_value(i, 'q1_how', 'how' in q1_words)
        data.set_value(i, 'q2_how', 'how' in q2_words)

    return data

In [3]:
train = extract_6w1h(train)
test = extract_6w1h(test)

In [4]:
print(',\n'.join(map(lambda s: "'{}'".format(s), train.columns)))

'id',
'qid1',
'qid2',
'question1',
'question2',
'is_duplicate',
'q1_what',
'q2_what',
'q1_when',
'q2_when',
'q1_why',
'q2_why',
'q1_where',
'q2_where',
'q1_who',
'q2_who',
'q1_which',
'q2_which',
'q1_how',
'q2_how'


In [5]:
features = [
    'q1_what',
    'q2_what',
    'q1_when',
    'q2_when',
    'q1_why',
    'q2_why',
    'q1_where',
    'q2_where',
    'q1_who',
    'q2_who',
    'q1_which',
    'q2_which',
    'q1_how',
    'q2_how'
]

In [8]:
for feature in features:
    train[feature] = train[feature].astype(np.bool)
    test[feature] = test[feature].astype(np.bool)

In [9]:
import util
util.save_feature(train, 'train', features, 'id')
util.save_feature(test, 'test', features, 'test_id')

In [15]:
ff = [
    'what',
    'when',
    'why',
    'where',
    'who',
    'which',
    'how'
]
for f in ff:
    train[f] = train['q1_' + f].astype(np.int8) + train['q2_' + f].astype(np.int8)
    test[f] = test['q1_' + f].astype(np.int8) + test['q2_' + f].astype(np.int8)
util.save_feature(train, 'train', ff, 'id')
util.save_feature(test, 'test', ff, 'test_id')